<div class="alert alert-success">
    <b>Sports Data Analytics</b> <br>
    instructed by Martin Rumo <br>
    as part of the study programme MScIDS <br>
    at the Lucerne University of Applied Sciences and Arts <br>
<br>
Created by Joël Lüscher and Noel Rinke | Submitted on the 27th of February 2023
</div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#FA-Women's-Super-League-Analysis" data-toc-modified-id="FA-Women's-Super-League-Analysis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>FA Women's Super League Analysis</a></span><ul class="toc-item"><li><span><a href="#Setup-&amp;-Configuration" data-toc-modified-id="Setup-&amp;-Configuration-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup &amp; Configuration</a></span></li></ul></li><li><span><a href="#Load-&amp;-Prepare-Data" data-toc-modified-id="Load-&amp;-Prepare-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load &amp; Prepare Data</a></span><ul class="toc-item"><li><span><a href="#Get-all-the-Match-Data-for-the-2018/19,-2019/20-&amp;-2020/21-Seasons" data-toc-modified-id="Get-all-the-Match-Data-for-the-2018/19,-2019/20-&amp;-2020/21-Seasons-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Get all the Match Data for the 2018/19, 2019/20 &amp; 2020/21 Seasons</a></span></li><li><span><a href="#Data-Overview-and-Query-Examples" data-toc-modified-id="Data-Overview-and-Query-Examples-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data Overview and Query Examples</a></span></li><li><span><a href="#Create-Match-Week-Data" data-toc-modified-id="Create-Match-Week-Data-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Create Match Week Data</a></span><ul class="toc-item"><li><span><a href="#Match-Data" data-toc-modified-id="Match-Data-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Match Data</a></span></li></ul></li></ul></li></ul></div>

# FA Women's Super League Analysis

This notebook includes the initial data preparation and an exploration of the data. It serves as the basis for the subsequent analysis of the 2018/19 season of Arsenal Women FC and the Chelsea Women FC.

The data is sourced from [StatsBomb](https://statsbomb.com/) and accessed using the StatsBomb API for which a function is implemented in the [mplsoccer](https://mplsoccer.readthedocs.io/en/latest/index.html) Python library. StatsBomb is a UK based sports analytics firm and the source of data for a lot of companies in the field of sports analytics. Further information about the StatsBomb data is provided in the [StatsBomb Open Data GitHub repository](https://github.com/statsbomb/open-data). The data dictionary for the events data, which is loaded and used for the analysis, can be found [here](https://github.com/statsbomb/open-data/blob/master/doc/Open%20Data%20Events%20v4.0.0.pdf).

Following this notebook, a Streamlit app is created which consists of separate modules for the data preparation and the visualizations to limit the code in the app.py script. The following image shows the content of this submission:

![image](img/sda_process.png)

## Setup & Configuration

In [1]:
!pip install statsbomb
!pip install mplsoccer

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsbomb as sb
from mplsoccer import Sbopen
from collections import defaultdict

In [3]:
# Pandas configuration
pd.options.display.precision = 2  # number of decimals shown
pd.options.display.max_rows = 100  # max number of rows shown
pd.options.display.max_columns = 100  # max number of columns shown
pd.options.display.max_colwidth = 60  # max width of cells

# Load & Prepare Data

The FA Women's Super League can be loaded using the competition_id 37. There are three seasons available for the FA Women's Super League as can be seen below.

In [4]:
comps = sb.Competitions()
json_data = comps.data  # underlying json data
df = comps.get_dataframe()
df[df['competition_id'] == 37]

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
15,37,90,England,FA Women's Super League,female,False,False,2020/2021,2023-02-25T14:52:09.326729,2021-06-13T16:17:31.694,None,2023-02-25T14:52:09.326729
16,37,42,England,FA Women's Super League,female,False,False,2019/2020,2023-02-25T12:45:18.376026,2021-06-13T16:17:31.694,None,2023-02-25T12:45:18.376026
17,37,4,England,FA Women's Super League,female,False,False,2018/2019,2022-12-01T11:41:40.025541,2021-06-13T16:17:31.694,None,2022-12-01T11:41:40.025541


## Get all the Match Data for the 2018/19, 2019/20 & 2020/21 Seasons

In the following two code cells, the data is downloaded, read as pandas dataframe and also saved as csv. A check is implemented whether the data was previously downloaded. If this is the case, the data is loaded from the directory.

In [5]:
def get_team_matches(parser, competition_id, season_id, team_name):
    ''' Get all the matches '''
    df_season = parser.match(competition_id=competition_id, season_id=season_id)
    df_team = df_season[(df_season['home_team_name'] == team_name) |
                      (df_season['away_team_name'] == team_name)]
    return df_team


def get_events_data(parser, match_files):
    ''' Get the events data based on the match ids'''
    parser = Sbopen()
    df_match_files = pd.concat([parser.event(file)[0] for file in match_files])
    return df_match_files


def add_match_date(df_events, df_season):
    ''' Add match data to the events '''
    df = df_events.merge(df_season[['match_id', 'match_week', 'match_date']], on='match_id')
    return df

In [6]:
import os.path
import os
import pandas as pd

folder_name = "data"

# Create the folder "data" if not exists
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
else:
    print(f"Folder '{folder_name}' already exists.")

# Instantiate a parser object
parser = Sbopen()

cfc_files_lst = ['./data/events_18_19_cfc.csv', './data/events_19_20_cfc.csv', './data/events_20_21_cfc.csv']
afc_files_lst = ['./data/events_18_19_afc.csv', './data/events_19_20_afc.csv', './data/events_20_21_afc.csv']
season_id_lst = [4, 42, 90]


def load_data(file, season_id, team_name, df_name_prefix, df_events_prefix, parser, extension=".csv"):
    if os.path.isfile(file):
        print(f"{file} exists.. start loading the data from csv")
        df = pd.read_csv(file)
        df_name = df_name_prefix + file.split('/')[-1].split('.')[0]
        globals()[df_name] = df
    else:
        print(f"{file} does not exist in directory.. start loading the data from Statsbomb")
        folder, file_name = os.path.split(file)
        file_name_without_extension, _ = os.path.splitext(file_name)
        result = file_name_without_extension.split("_")[-3:]
        season_team = "_".join(result)
        df = get_team_matches(parser, competition_id=37, season_id=season_id, team_name=team_name)
        df_name = df_events_prefix + season_team
        df_events = get_events_data(parser, match_files=df['match_id'].to_list())
        df_events = add_match_date(df_events, df)
        csv_name = df_name[len(df_events_prefix):]
        df_events.to_csv("./data/events_" + csv_name + extension, index=False)
        df_name = df_name_prefix + file.split('/')[-1].split('.')[0]
        globals()[df_name] = df_events

# Load Chelsea FCW events data
for file, season_id in zip(cfc_files_lst, season_id_lst):
    load_data(file, season_id, 'Chelsea FCW', "df_", "df_events_", parser)

# Load Arsenal WFC events data
for file, season_id in zip(afc_files_lst, season_id_lst):
    load_data(file, season_id, 'Arsenal WFC', "df_", "df_events_", parser)

Folder 'data' already exists.
./data/events_18_19_cfc.csv exists.. start loading the data from csv


C:\Users\NoelRinke\AppData\Local\Temp\ipykernel_20836\3057302906.py:24: DtypeWarning: Columns (52,56,57,58,59,62,65,66,69,70,73,74,75,78,79,80,81,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


./data/events_19_20_cfc.csv exists.. start loading the data from csv


C:\Users\NoelRinke\AppData\Local\Temp\ipykernel_20836\3057302906.py:24: DtypeWarning: Columns (41,55,57,59,60,61,62,65,68,69,71,72,73,75,77,79,80,81,82,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


./data/events_20_21_cfc.csv exists.. start loading the data from csv


...

C:\Users\NoelRinke\AppData\Local\Temp\ipykernel_20836\3057302906.py:24: DtypeWarning: Columns (54,55,59,60,63,64,70,71,72,75,76,78,79,80,82,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


./data/events_19_20_afc.csv exists.. start loading the data from csv


C:\Users\NoelRinke\AppData\Local\Temp\ipykernel_20836\3057302906.py:24: DtypeWarning: Columns (59,61,62,66,69,70,74,75,76,77,78,79,80,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


./data/events_20_21_afc.csv exists.. start loading the data from csv


C:\Users\NoelRinke\AppData\Local\Temp\ipykernel_20836\3057302906.py:24: DtypeWarning: Columns (56,61,62,63,69,70,73,75,76,78,79,80,81,82,83,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Having downloaded the events dataframes, we would like to check if the data was imported successfully and if the number of match weeks is correct. The number of matches played by each team varied in the three seasons analysed in this report. The 2018/19 season was played with 11 teams following the change to a fully professional league for the first time. The 2019/20 season was halted after 15 matches due to the Covid-19 pandemic.

In [7]:
# Check if all match days are available
assert len(df_events_18_19_cfc['match_week'].unique()) == 20, f"Number of league matches does not correspond to 20, got: {len(df_season_18_19_cfc)}"
assert len(df_events_19_20_cfc['match_week'].unique()) == 15, f"Number of league matches does not correspond to 15, got: {len(df_season_19_20_cfc)}"
assert len(df_events_20_21_cfc['match_week'].unique()) == 22, f"Number of league matches does not correspond to 22, got: {len(df_season_20_21_cfc)}"

In [8]:
df_events_20_21_cfc.head(1)

,id,index,period,timestamp,minute,second,possession,duration,match_id,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,end_x,end_y,body_part_id,body_part_name,sub_type_id,sub_type_name,x,y,outcome_id,outcome_name,under_pressure,out,pass_cross,aerial_won,counterpress,pass_switch,off_camera,technique_id,technique_name,ball_recovery_recovery_failure,shot_statsbomb_xg,goalkeeper_position_id,goalkeeper_position_name,pass_assisted_shot_id,pass_shot_assist,shot_key_pass_id,end_z,shot_redirect,shot_first_time,injury_stoppage_in_chain,block_offensive,pass_goal_assist,dribble_overrun,ball_recovery_offensive,dribble_nutmeg,substitution_replacement_id,substitution_replacement_name,foul_won_defensive,foul_committed_advantage,foul_won_advantage,pass_miscommunication,pass_cut_back,foul_committed_card_id,foul_committed_card_name,shot_one_on_one,dribble_no_touch,pass_deflected,shot_open_goal,foul_committed_offensive,pass_no_touch,bad_behaviour_card_id,bad_behaviour_card_name,block_deflection,block_save_block,foul_committed_penalty,foul_won_penalty,shot_deflected,shot_follows_dribble,match_week,match_date
0,e89618d8-f743-4c69-b267-8d06ae645927,1,1,00:00:00,0,0,1,0.0,3775581,35,Starting XI,971,Chelsea FCW,1,Regular Play,971,Chelsea FCW,433.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,2021-03-28


In [9]:
# Check if all match days are available
assert len(df_events_18_19_afc['match_week'].unique()) == 20, f"Number of league matches does not correspond to 20, got: {len(df_season_18_19_afc)}"
assert len(df_events_19_20_afc['match_week'].unique()) == 15, f"Number of league matches does not correspond to 15, got: {len(df_season_19_20_afc)}"
assert len(df_events_20_21_afc['match_week'].unique()) == 22, f"Number of league matches does not correspond to 22, got: {len(df_season_20_21_afc)}"

In [10]:
df_events_18_19_cfc['match_week']

0         0
1         0
2         0
3         0
4         0
         ..
68958    16
68959    16
68960    16
68961    16
68962    16
Name: match_week, Length: 68963, dtype: int64

In [11]:
df_events_20_21_afc.head(1)

,id,index,period,timestamp,minute,second,possession,duration,match_id,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,end_x,end_y,body_part_id,body_part_name,sub_type_id,sub_type_name,x,y,under_pressure,outcome_id,outcome_name,counterpress,pass_cross,aerial_won,pass_assisted_shot_id,pass_shot_assist,shot_statsbomb_xg,shot_key_pass_id,technique_id,technique_name,goalkeeper_position_id,goalkeeper_position_name,out,off_camera,pass_switch,pass_cut_back,ball_recovery_recovery_failure,foul_won_defensive,pass_deflected,end_z,foul_committed_advantage,foul_won_advantage,shot_first_time,block_offensive,pass_no_touch,pass_miscommunication,pass_goal_assist,substitution_replacement_id,substitution_replacement_name,block_deflection,dribble_nutmeg,shot_one_on_one,shot_open_goal,foul_committed_card_id,foul_committed_card_name,injury_stoppage_in_chain,dribble_overrun,foul_committed_penalty,foul_won_penalty,bad_behaviour_card_id,bad_behaviour_card_name,foul_committed_offensive,shot_follows_dribble,ball_recovery_offensive,block_save_block,shot_redirect,shot_deflected,dribble_no_touch,match_week,match_date
0,f51b1630-d1a8-4837-97b0-de862f0e299a,1,1,00:00:00,0,0,1,0.0,3775648,35,Starting XI,2647,Aston Villa,1,Regular Play,2647,Aston Villa,352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,2021-02-28


## Data Overview and Query Examples

In [12]:
df_events_18_19_cfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68963 entries, 0 to 68962
Data columns (total 87 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              68963 non-null  object 
 1   index                           68963 non-null  int64  
 2   period                          68963 non-null  int64  
 3   timestamp                       68963 non-null  object 
 4   minute                          68963 non-null  int64  
 5   second                          68963 non-null  int64  
 6   possession                      68963 non-null  int64  
 7   duration                        50977 non-null  float64
 8   match_id                        68963 non-null  int64  
 9   type_id                         68963 non-null  int64  
 10  type_name                       68963 non-null  object 
 11  possession_team_id              68963 non-null  int64  
 12  possession_team_name            

In [13]:
df_events_18_19_cfc[df_events_18_19_cfc['player_name'].notna() & 
                    df_events_18_19_cfc['player_name'].str.contains('Kirby')].head(1)

,id,index,period,timestamp,minute,second,possession,duration,match_id,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,end_x,end_y,body_part_id,body_part_name,sub_type_id,sub_type_name,x,y,under_pressure,outcome_id,outcome_name,pass_assisted_shot_id,pass_shot_assist,technique_id,technique_name,shot_statsbomb_xg,shot_key_pass_id,goalkeeper_position_id,goalkeeper_position_name,pass_cross,counterpress,ball_recovery_recovery_failure,pass_switch,shot_first_time,pass_goal_assist,end_z,dribble_nutmeg,aerial_won,dribble_overrun,pass_backheel,pass_deflected,block_deflection,substitution_replacement_id,substitution_replacement_name,shot_one_on_one,foul_committed_advantage,foul_won_advantage,injury_stoppage_in_chain,foul_committed_offensive,foul_won_defensive,bad_behaviour_card_id,bad_behaviour_card_name,ball_recovery_offensive,pass_cut_back,off_camera,foul_won_penalty,block_offensive,shot_deflected,foul_committed_card_id,foul_committed_card_name,pass_miscommunication,block_save_block,shot_open_goal,foul_committed_penalty,out,pass_no_touch,shot_redirect,match_week,match_date
34,f55b153d-56db-4ccc-8117-d90185576727,34,1,00:00:42.113000,0,42,4,NaN,7298,42,Ball Receipt,971,Chelsea FCW,6,From Counter,971,Chelsea FCW,NaN,4641.0,Francesca Kirby,22.0,Right Center Forward,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018-02-24


In [14]:
df_events_18_19_cfc[df_events_18_19_cfc['outcome_name'].notna() & 
                    df_events_18_19_cfc['outcome_name'].str.contains('Goal')].head(1)

,id,index,period,timestamp,minute,second,possession,duration,match_id,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,end_x,end_y,body_part_id,body_part_name,sub_type_id,sub_type_name,x,y,under_pressure,outcome_id,outcome_name,pass_assisted_shot_id,pass_shot_assist,technique_id,technique_name,shot_statsbomb_xg,shot_key_pass_id,goalkeeper_position_id,goalkeeper_position_name,pass_cross,counterpress,ball_recovery_recovery_failure,pass_switch,shot_first_time,pass_goal_assist,end_z,dribble_nutmeg,aerial_won,dribble_overrun,pass_backheel,pass_deflected,block_deflection,substitution_replacement_id,substitution_replacement_name,shot_one_on_one,foul_committed_advantage,foul_won_advantage,injury_stoppage_in_chain,foul_committed_offensive,foul_won_defensive,bad_behaviour_card_id,bad_behaviour_card_name,ball_recovery_offensive,pass_cut_back,off_camera,foul_won_penalty,block_offensive,shot_deflected,foul_committed_card_id,foul_committed_card_name,pass_miscommunication,block_save_block,shot_open_goal,foul_committed_penalty,out,pass_no_touch,shot_redirect,match_week,match_date
256,3f0fc8e9-a09f-480a-9396-132e1ca05ec5,256,1,00:05:46.380000,5,46,16,1.48,7298,16,Shot,971,Chelsea FCW,2,From Corner,971,Chelsea FCW,NaN,4642.0,Millie Bright,3.0,Right Center Back,NaN,NaN,NaN,NaN,NaN,NaN,120.0,43.2,38.0,Left Foot,87.0,Open Play,108.0,32.0,1.0,97.0,Goal,NaN,NaN,93.0,Normal,0.07,e3effe10-23f0-413d-81e3-a468b90091b2,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018-02-24


In [15]:
# Group the data by match_week and count the number of goals for each match_week
df_events_19_20_cfc[df_events_18_19_cfc["outcome_name"] == "Goal"].groupby("match_week")["type_name"].count().to_dict()

{1: 5,
 2: 1,
 3: 3,
 4: 1,
 5: 1,
 6: 3,
 7: 4,
 9: 3,
 10: 4,
 11: 2,
 12: 2,
 13: 3,
 14: 3,
 16: 2,
 17: 6}

In [16]:
df_events_18_19_cfc[(df_events_18_19_cfc['type_name'] == 'Pass')].head(1)

,id,index,period,timestamp,minute,second,possession,duration,match_id,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,end_x,end_y,body_part_id,body_part_name,sub_type_id,sub_type_name,x,y,under_pressure,outcome_id,outcome_name,pass_assisted_shot_id,pass_shot_assist,technique_id,technique_name,shot_statsbomb_xg,shot_key_pass_id,goalkeeper_position_id,goalkeeper_position_name,pass_cross,counterpress,ball_recovery_recovery_failure,pass_switch,shot_first_time,pass_goal_assist,end_z,dribble_nutmeg,aerial_won,dribble_overrun,pass_backheel,pass_deflected,block_deflection,substitution_replacement_id,substitution_replacement_name,shot_one_on_one,foul_committed_advantage,foul_won_advantage,injury_stoppage_in_chain,foul_committed_offensive,foul_won_defensive,bad_behaviour_card_id,bad_behaviour_card_name,ball_recovery_offensive,pass_cut_back,off_camera,foul_won_penalty,block_offensive,shot_deflected,foul_committed_card_id,foul_committed_card_name,pass_miscommunication,block_save_block,shot_open_goal,foul_committed_penalty,out,pass_no_touch,shot_redirect,match_week,match_date
5,2a456ec2-352c-499b-b5cc-e68bf84c7e9a,5,1,00:00:00.100000,0,0,2,0.0,7298,30,Pass,971,Chelsea FCW,9,From Kick Off,971,Chelsea FCW,NaN,4647.0,So-Yun Ji,14.0,Center Midfield,4659.0,Ramona Bachmann,3.61,-0.98,1.0,Ground Pass,63.0,37.0,40.0,Right Foot,65.0,Kick Off,61.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018-02-24


In [17]:
df_events_18_19_cfc['outcome_name'].unique()

array([nan, 'Incomplete', 'Complete', 'Blocked', 'Lost Out',
       'In Play Safe', 'Lost In Play', 'Won', 'Success In Play', 'Goal',
       'No Touch', 'Out', 'Pass Offside', 'Clear', 'Off T', 'Saved',
       'In Play Danger', 'Unknown', 'Tactical', 'Success', 'Touched Out',
       'Wayward', 'Claim', 'Saved Twice', 'Touched In', 'Success Out',
       'Injury', 'Post', 'Lost', 'Success To Team', 'Injury Clearance',
       'Success To Opposition', 'Fail', 'Punched out', 'Collected Twice'],
      dtype=object)

In [18]:
df_events_18_19_cfc['type_name'].unique()

array(['Starting XI', 'Half Start', 'Ball Receipt', 'Pass', 'Carry',
       'Pressure', 'Miscontrol', 'Block', 'Ball Recovery',
       'Dribbled Past', 'Dribble', 'Shot', 'Goal Keeper', 'Clearance',
       'Duel', 'Dispossessed', 'Camera On', 'Interception', 'Shield',
       'Foul Committed', 'Foul Won', 'Offside', 'Tactical Shift',
       'Injury Stoppage', 'Player Off', 'Player On', 'Half End',
       'Substitution', 'Error', 'Bad Behaviour', 'Camera off',
       'Referee Ball-Drop', '50/50'], dtype=object)

In [19]:
# use rank to give each date a number that increases with the date
df_events_18_19_cfc['date_number'] = df_events_18_19_cfc['match_date'].rank(method='dense').astype(int)
df_events_18_19_afc['date_number'] = df_events_18_19_afc['match_date'].rank(method='dense').astype(int)

In [20]:
len(df_events_18_19_afc['date_number'].unique())

20

## Create Match Week Data

In the following, the outcome_name and type_name of the events dataframes are used to aggregate the data for each match week and both teams.

In [21]:
def get_match_week_stat(df, team_name, variable):
    ''' Returns a list with the specified outcome count for each match week.
        If the outcome_name does not occur, the value 0 is added as value to the match_week. '''
    match_weeks = range(1, df["date_number"].max() + 1)
    dct = defaultdict(int)
    for match_week in match_weeks:
        dct[match_week-1] = df[(df["date_number"] == match_week) &
                             (df["team_name"] == team_name) &
                             ((df["outcome_name"] == variable) | (df["type_name"] == variable))].shape[0]
    df = pd.DataFrame.from_dict(dict(dct), orient='index', columns=[variable])
    return df[variable].tolist()


def get_match_week_stat_op(df, team_name, variable):
    ''' Returns a list with the specified outcome count for each match week for the opposing team.
        If the outcome_name does not occur, the value 0 is added as value to the match_week. '''
    match_weeks = range(df["date_number"].min(), df["date_number"].max() + 1)
    dct = defaultdict(int)
    for match_week in match_weeks:
        dct[match_week-1] = df[(df["date_number"] == match_week) &
                             (df["team_name"] != team_name) & ((df["outcome_name"] == variable) | (df["type_name"] == variable))].shape[0]
    df = pd.DataFrame.from_dict(dict(dct), orient='index', columns=[variable])
    return df[variable].tolist()


def get_match_week_stat_nested_calc(df, team_name, event_type, var_type, calc):
    if calc == 'count':
        return df[(df['type_name'] == event_type) &
                  (df['team_name'] == team_name)].groupby('date_number').count()[var_type].tolist()
    elif calc == 'sum':
        return df[(df['type_name'] == event_type) & 
                  (df['team_name'] == team_name)].groupby('date_number').sum()[var_type].tolist()
    elif calc == 'mean':
        return df[(df['type_name'] == event_type) &
                  (df['team_name'] == team_name)].groupby('date_number').mean()[var_type].tolist()


def generate_match_week_df(df, team_name):
    goals_lst = get_match_week_stat(df, team_name, 'Goal')
    goals_conceded_lst = get_match_week_stat_op(df, team_name, 'Goal')
    shots_lst = get_match_week_stat(df, team_name, 'Shot')
    off_target_lst = get_match_week_stat(df, team_name, 'Off T')
    shot_blocked_lst = get_match_week_stat(df, team_name, 'Blocked')
    shots_saved_lst = get_match_week_stat(df, team_name, 'Saved')
    xg_lst = df.groupby('date_number').sum()['shot_statsbomb_xg'].to_list()
    subs_lst = get_match_week_stat(df, team_name, 'Substitution')
    offs_lst = get_match_week_stat(df, team_name, 'Offside')
    clear_lst = get_match_week_stat(df, team_name, 'Clearance')
    pass_length_lst = get_match_week_stat_nested_calc(df, team_name, 'Pass', 'pass_length', 'sum')
    pass_length_avg_lst = get_match_week_stat_nested_calc(df, team_name, 'Pass', 'pass_length', 'mean')
    pass_cnt_lst = get_match_week_stat_nested_calc(df, team_name, 'Pass', 'pass_length', 'count')
    matches_numbered = df['date_number'].to_list()

    df = pd.DataFrame(list(zip(goals_lst, goals_conceded_lst, shots_lst, off_target_lst, shot_blocked_lst,
                               shots_saved_lst, xg_lst, subs_lst, offs_lst, clear_lst, pass_length_lst,
                               pass_length_avg_lst, pass_cnt_lst)),
                      columns=['GoalsScored', 'GoalsConceded', 'Shots', 'ShotOffT', 'ShotsBlocked', 'ShotsSaved',
                               'ShotXG', 'Substitutions', 'Offsides', 'Clearances', 'PassLengthSum', 'PassLengthAvg',
                               'PassCnt'])

    df = df.reset_index()
    df.rename(columns={'index': 'MatchWeek'}, inplace=True)
    df = df.set_index('MatchWeek')
    df.index = df.index + 1
    return df

In [22]:
df_match_week_cfc_18_19 = generate_match_week_df(df_events_18_19_cfc, 'Chelsea FCW')
df_match_week_afc_18_19 = generate_match_week_df(df_events_18_19_afc, 'Arsenal WFC')

In [23]:
df_match_week_cfc_18_19

,GoalsScored,GoalsConceded,Shots,ShotOffT,ShotsBlocked,ShotsSaved,ShotXG,Substitutions,Offsides,Clearances,PassLengthSum,PassLengthAvg,PassCnt
MatchWeek,,,,,,,,,,,,,
1,2,2,25,7,11,5,3.98,3,4,10,10887.19,20.90,521
2,0,0,15,5,4,6,1.91,3,0,5,9907.90,22.99,431
3,0,0,25,9,4,7,1.50,2,0,0,16123.30,22.03,732
4,0,0,20,8,4,6,2.69,3,0,6,13774.37,23.39,589
5,2,0,24,9,6,7,3.08,3,0,4,14484.77,22.53,643
6,0,5,13,6,2,5,3.15,3,0,12,12442.04,23.04,540
7,0,0,22,6,7,8,2.74,3,0,10,9041.84,21.95,412
8,1,0,22,8,7,5,1.81,3,0,20,14617.37,22.15,660
9,2,0,15,3,6,3,2.23,3,0,18,11964.48,23.23,515


### Match Data

The `get_team_matches` function created above is used to get the match_id which can then be used to extract statistics for a certain match. The dataframe below shows the 2018-19 season of Chelsea FCW which contains information about the matches the team played.

In [24]:
df_season_cfc_18_19 = get_team_matches(parser, competition_id=37, season_id=4, team_name='Chelsea FCW')
df_season_cfc_18_19

,match_id,match_date,kick_off,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_id,country_name,competition_name,season_id,season_name,home_team_id,home_team_name,home_team_gender,home_team_group,home_team_country_id,home_team_country_name,home_team_managers_id,home_team_managers_name,home_team_managers_nickname,home_team_managers_dob,home_team_managers_country_id,home_team_managers_country_name,away_team_id,away_team_name,away_team_gender,away_team_group,away_team_country_id,away_team_country_name,away_team_managers_id,away_team_managers_name,away_team_managers_nickname,away_team_managers_dob,away_team_managers_country_id,away_team_managers_country_name,metadata_data_version,competition_stage_id,competition_stage_name,stadium_id,stadium_name,stadium_country_id,stadium_country_name,referee_id,referee_name,referee_country_id,referee_country_name,metadata_shot_fidelity_version
0,7298,2018-02-24,2018-02-24 20:45:00,2,2,available,scheduled,2022-05-27 12:02:24.272045,2021-06-13 16:17:31.694,0,37,England,FA Women's Super League,4,2018/2019,746,Manchester City WFC,female,None,68,England,30,Nick Cushing,Nick Cushing,1984-11-09,68,England,971,Chelsea FCW,female,None,68,England,152,Emma Hayes,Emma Hayes,1976-10-18,68,England,1.0.2,1,Regular Season,99,Academy Stadium,68,England,109,Darren England,68,England,NaN
3,19736,2018-10-14,2018-10-14 15:00:00,0,5,available,scheduled,2020-07-29 05:00:00.000000,2021-06-13 16:17:31.694,5,37,England,FA Women's Super League,4,2018/2019,971,Chelsea FCW,female,None,68,England,152,Emma Hayes,Emma Hayes,1976-10-18,68,England,968,Arsenal WFC,female,None,68,England,31,Joseph Montemurro,Joseph Montemurro,1969-09-13,14,Australia,1.0.3,1,Regular Season,132,Kingsmeadow,68,England,905,Sam Purkiss,68,England,NaN
5,19772,2018-12-02,2018-12-02 15:00:00,1,0,available,scheduled,2020-07-29 05:00:00.000000,2021-06-13 16:17:31.694,11,37,England,FA Women's Super League,4,2018/2019,971,Chelsea FCW,female,None,68,England,152,Emma Hayes,Emma Hayes,1976-10-18,68,England,974,Reading WFC,female,None,68,England,144,Kelly Chambers,Kelly Chambers,1986-01-11,68,England,1.0.3,1,Regular Season,132,Kingsmeadow,68,England,894,Lucy Oliver,68,England,NaN
6,19730,2018-09-30,2018-09-30 15:00:00,2,0,available,scheduled,2020-07-29 05:00:00.000000,2021-06-13 16:17:31.694,4,37,England,FA Women's Super League,4,2018/2019,971,Chelsea FCW,female,None,68,England,152,Emma Hayes,Emma Hayes,1976-10-18,68,England,965,Brighton & Hove Albion WFC,female,None,68,England,149,Hope Patricia Powell,Hope Powell,1966-12-08,68,England,1.0.3,1,Regular Season,132,Kingsmeadow,68,England,895,Abigail Marriott,68,England,NaN
7,19794,2019-02-20,2019-02-20 20:30:00,6,0,available,scheduled,2020-07-29 05:00:00.000000,2021-06-13 16:17:31.694,17,37,England,FA Women's Super League,4,2018/2019,971,Chelsea FCW,female,None,68,England,152,Emma Hayes,Emma Hayes,1976-10-18,68,England,973,Bristol City WFC,female,None,68,England,143,Tanya Oxtoby,Tanya Oxtoby,1982-06-15,14,Australia,1.0.3,1,Regular Season,132,Kingsmeadow,68,England,932,Tom Bishop,68,England,NaN
8,19820,2019-05-11,2019-05-11 13:30:00,2,3,available,scheduled,2020-07-29 05:00:00.000000,2021-06-13 16:17:31.694,22,37,England,FA Women's Super League,4,2018/2019,974,Reading WFC,female,None,68,England,144,Kelly Chambers,Kelly Chambers,1986-01-11,68,England,971,Chelsea FCW,female,None,68,England,152,Emma Hayes,Emma Hayes,1976-10-18,68,England,1.1.0,1,Regular Season,577,Adams Park,68,England,937,Abigail Byrne,68,England,2
14,19743,2018-10-21,2018-10-21 13:30:00,0,0,available,scheduled,2021-04-28 07:08:31.946271,2021-06-13 16:17:31.694,6,37,England,FA Women's Super League,4,2018/2019,969,Birmingham City WFC,female,None,68,England,128,Marc Skinner,Marc Skinner,1983-03-23,68,England,971,Chelsea FCW,female,None,68,England,152,Emma Hayes,Emma Hayes,1976-10-18,68,England,1.0.3,1,Regular Season,5332,\tARMCO Arena,68,England,898,Amy Fearn,68,England,NaN
19,19748,2018-10-28,2018-10

In [25]:
df_season_afc_18_19 = get_team_matches(parser, competition_id=37, season_id=4, team_name='Arsenal WFC')
df_season_afc_18_19

,match_id,match_date,kick_off,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_id,country_name,competition_name,season_id,season_name,home_team_id,home_team_name,home_team_gender,home_team_group,home_team_country_id,home_team_country_name,home_team_managers_id,home_team_managers_name,home_team_managers_nickname,home_team_managers_dob,home_team_managers_country_id,home_team_managers_country_name,away_team_id,away_team_name,away_team_gender,away_team_group,away_team_country_id,away_team_country_name,away_team_managers_id,away_team_managers_name,away_team_managers_nickname,away_team_managers_dob,away_team_managers_country_id,away_team_managers_country_name,metadata_data_version,competition_stage_id,competition_stage_name,stadium_id,stadium_name,stadium_country_id,stadium_country_name,referee_id,referee_name,referee_country_id,referee_country_name,metadata_shot_fidelity_version
3,19736,2018-10-14,2018-10-14 15:00:00,0,5,available,scheduled,2020-07-29 05:00:00.000000,2021-06-13 16:17:31.694,5,37,England,FA Women's Super League,4,2018/2019,971,Chelsea FCW,female,None,68,England,152,Emma Hayes,Emma Hayes,1976-10-18,68,England,968,Arsenal WFC,female,None,68,England,31,Joseph Montemurro,Joseph Montemurro,1969-09-13,14,Australia,1.0.3,1,Regular Season,132,Kingsmeadow,68,England,905,Sam Purkiss,68,England,NaN
9,19766,2018-11-25,2018-11-25 13:30:00,4,1,available,scheduled,2020-07-29 05:00:00.000000,2021-06-13 16:17:31.694,10,37,England,FA Women's Super League,4,2018/2019,968,Arsenal WFC,female,None,68,England,31,Joseph Montemurro,Joseph Montemurro,1969-09-13,14,Australia,965,Brighton & Hove Albion WFC,female,None,68,England,149,Hope Patricia Powell,Hope Powell,1966-12-08,68,England,1.0.3,1,Regular Season,456,Meadow Park,68,England,915,Rob Whitton,68,England,NaN
11,19717,2018-09-09,2018-09-09 13:30:00,5,0,available,scheduled,2020-07-29 05:00:00.000000,2021-06-13 16:17:31.694,1,37,England,FA Women's Super League,4,2018/2019,968,Arsenal WFC,female,None,68,England,31,Joseph Montemurro,Joseph Montemurro,1969-09-13,14,Australia,966,Liverpool WFC,female,None,68,England,154,Neil Redfearn,Neil Redfearn,1965-06-20,68,England,1.0.3,1,Regular Season,456,Meadow Park,68,England,569,Jason Richardson,68,England,NaN
17,19800,2019-03-14,2019-03-14 20:30:00,4,0,available,scheduled,2020-08-24 14:34:34.401523,2021-06-13 16:17:31.694,18,37,England,FA Women's Super League,4,2018/2019,968,Arsenal WFC,female,None,68,England,31,Joseph Montemurro,Joseph Montemurro,1969-09-13,14,Australia,973,Bristol City WFC,female,None,68,England,143,Tanya Oxtoby,Tanya Oxtoby,1982-06-15,14,Australia,1.1.0,1,Regular Season,456,Meadow Park,68,England,915,Rob Whitton,68,England,NaN
21,19785,2019-01-13,2019-01-13 13:30:00,1,2,available,scheduled,2020-07-29 05:00:00.000000,2021-06-13 16:17:31.694,14,37,England,FA Women's Super League,4,2018/2019,968,Arsenal WFC,female,None,68,England,31,Joseph Montemurro,Joseph Montemurro,1969-09-13,14,Australia,971,Chelsea FCW,female,None,68,England,152,Emma Hayes,Emma Hayes,1976-10-18,68,England,1.0.3,1,Regular Season,456,Meadow Park,68,England,566,Rebecca Welch,68,England,NaN
22,19749,2018-11-04,2018-11-04 13:30:00,3,1,available,scheduled,2021-04-28 19:48:01.152793,2021-06-13 16:17:31.694,8,37,England,FA Women's Super League,4,2018/2019,968,Arsenal WFC,female,None,68,England,31,Joseph Montemurro,Joseph Montemurro,1969-09-13,14,Australia,969,Birmingham City WFC,female,None,68,England,128,Marc Skinner,Marc Skinner,1983-03-23,68,England,1.0.3,1,Regular Season,456,Meadow Park,68,England,566,Rebecca Welch,68,England,NaN
29,19770,2018-12-02,2018-12-02 13:30:00,2,0,available,scheduled,2020-07-29 05:00:00.000000,2021-06-13 16:17:31.694,11,37,England,FA Women's Super League,4,2018/2019,746,Manchester City WFC,female,None,68,England,30,Nick Cushing,Nick Cushing,1984-11-09,68,England,968,Arsenal WFC,female,None,68,England,31,Joseph Montemurro,Joseph Montemurro,1969-09-13,14,Australia,1.0.3,1,Regular Seaso